<a href="https://colab.research.google.com/github/lblabucsd/Zada_Schulze_24/blob/main/dc_suite2p_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install/import outside packages
!pip install "opencv-python-headless<4.3"
!pip install suite2p
!pip install pynrrd
!pip install paramiko
import os, requests
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
from tifffile import imread
import scipy
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
import suite2p
import nrrd


# Figure Style settings for notebook.
import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (18, 13),
    'ytick.major.left': True,
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

from google.colab import drive
drive.mount('/content/drive')


AttributeError: ignored

## Set pipeline parameters

You can find an explanation of each op parameters [here](https://suite2p.readthedocs.io/en/latest/settings.html).

In [ ]:
ops = suite2p.default_ops()

ops['nchannels'] = 1
ops['do_bidiphase'] = True
ops['keepdims'] = False
ops['sparse_mode'] = False
ops['spatial_scale'] = 1
ops['snr_thresh'] = 1.2
ops['maxregshiftNR'] = 5
ops['high_pass'] = 50
ops['max_overlap'] = 0.7
ops['neuropil_extract'] = False
ops['inner_neuropil_radius'] = 1
ops['soma_crop'] = False
ops['batch_size'] = 100 # we will decrease the batch_size in case low RAM on computer
ops['threshold_scaling'] = 0.6 # we are increasing the threshold for finding ROIs to limit the number of non-cell ROIs found (sometimes useful in gcamp injections)

"""
#cellpose features
ops['anatomical_only'] = 1 # run cellpose to get masks on 1: max_proj / mean_img; 2: mean_img; 3: mean_img enhanced, 4: max_proj
ops['diameter'] = 5  # use diameter for cellpose, if 0 estimate diameter
ops['cellprob_threshold'] =  0.0 # cellprob_threshold for cellpose
ops['flow_threshold'] = 1.1 # flow_threshold for cellpose
ops['spatial_hp_cp'] = 25 # high-pass image spatially by a multiple of the diameter
ops['pretrained_model'] = 'nuclei' # path to pretrained model or model type string in Cellpose (can be user model)
"""
print(ops)


{'suite2p_version': '0.13.0', 'look_one_level_down': False, 'fast_disk': [], 'delete_bin': False, 'mesoscan': False, 'bruker': False, 'bruker_bidirectional': False, 'h5py': [], 'h5py_key': 'data', 'nwb_file': '', 'nwb_driver': '', 'nwb_series': '', 'save_path0': [], 'save_folder': [], 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 10.0, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': False, 'ignore_flyback': [], 'preclassify': 0.0, 'save_mat': False, 'save_NWB': False, 'combined': True, 'aspect': 1.0, 'do_bidiphase': True, 'bidiphase': 0, 'bidi_corrected': False, 'do_registration': True, 'two_step_registration': False, 'keep_movie_raw': False, 'nimg_init': 300, 'batch_size': 100, 'maxregshift': 0.1, 'align_by_chan': 1, 'reg_tif': False, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma_time': 0, 'smooth_sigma': 1.15, 'th_badframes': 1.0, 'norm_frames': True, 'force_refImg': False, 'pad_fft': False, 'non

Load data (select one fish at a time to be "f")

In [ ]:
subjects=['DZ-230518_wk2_f1_z-170_schooling'] #note: check 6 wk and 4 week from 5/03 and 4 (now with better frame rate estimate)
f=subjects[0]


#path = '/content/drive/MyDrive/Temporary_data_storage/Matt/Dc_imaging_April2023/'+f+'/'
path = '/content/drive/MyDrive/Personal_notebooks/David_Notebook/2p_imaging/'+f[3:9]+'/'+f+'/'

tree = ET.parse(path+"Experiment.xml")
root = tree.getroot()

pix_size = float(root[12].attrib['pixelSizeUM'])
frame_rate = (root[12].attrib['frameRate'])
frame_avg = (root[12].attrib['averageNum'])
z_steps = 1
z_flyback = 0
#z_steps = (root[8].attrib['steps'])
#z_flyback = (root[8].attrib['zStreamFrames'])
timepoints = (root[9].attrib['timepoints'])

print("pixels are", pix_size, "um, and volumes of ", int(z_steps)-int(z_flyback), " steps (",int(z_flyback)," flyback) aquired at ", float(frame_rate)/float(z_steps*frame_avg), " Hz, with ", timepoints, " timepoints" )

'''
#explore xml data here
for ndx,child in enumerate(root):
    print(ndx, child.tag)
    print(child.attrib)
'''

#use xml data to set additional params
ops['nplanes'] = int(z_steps)
ops['ignore_flyback'] = [int(z_flyback)-1]
ops['fs'] = float(frame_rate)/float(z_steps*frame_avg)
ops['tau'] = 3.0 #H2B-GC6s
ops['diameter'] = int(7.0/pix_size)

print('cell diameter in pixels is ', ops['diameter'])

pixels are 0.895 um, and volumes of  1  steps ( 0  flyback) aquired at  6.48  Hz, with  3500  timepoints
cell diameter in pixels is  7


In [ ]:
#RUN ALL as pipeline
#note: if registration already done, it won't re-run that

db = {'data_path': [path],}
print(db)

#!mkdir path+"test/"
output_ops = suite2p.run_s2p(ops=ops, db=db)

print(set(output_ops.keys()).difference(ops.keys()))
list(Path(output_ops['save_path']).iterdir())
output_ops_file = np.load(Path(output_ops['save_path']).joinpath('ops.npy'), allow_pickle=True).item()
output_ops_file.keys() == output_ops.keys()


{'data_path': ['/content/drive/MyDrive/Personal_notebooks/David_Notebook/2p_imaging/230518/DZ-230518_wk2_f1_z-170_schooling/']}
{'data_path': ['/content/drive/MyDrive/Personal_notebooks/David_Notebook/2p_imaging/230518/DZ-230518_wk2_f1_z-170_schooling/']}
tif
** Found 1 tifs - converting to binary **
NOTE: ScanImageTiffReader not installed, using tifffile
400 frames of binary, time 9.33 sec.
800 frames of binary, time 14.63 sec.
1200 frames of binary, time 20.24 sec.
1600 frames of binary, time 31.12 sec.
2000 frames of binary, time 44.76 sec.
2400 frames of binary, time 60.45 sec.
2800 frames of binary, time 87.09 sec.
3200 frames of binary, time 110.29 sec.
time 116.46 sec. Wrote 3500 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
NOTE: Applying builtin classifier at /usr/local/lib/python3.10/dist-packages/suite2p/classifiers/classifier.n

True

In [ ]:
#check overlap of early and late frames
#if they dont overlap well, then discard (likley z drift)
import suite2p
from suite2p.io import BinaryFile

plt.figure(figsize=[10,5])
plt.subplot(121)
with BinaryFile(Ly=output_ops['Ly'],
                Lx=output_ops['Lx'],
                filename=output_ops['reg_file']) as f:
    plt.imshow(f[0], cmap='GnBu', alpha=0.5, vmin=20, vmax=150)
    plt.imshow(f[2100], cmap='OrRd', alpha=0.5, vmin=20, vmax=150)
plt.subplot(122)
with BinaryFile(Ly=output_ops['Ly'],
                Lx=output_ops['Lx'],
                filename=output_ops['reg_file']) as f:
    plt.imshow(f[0], cmap='GnBu', alpha=0.5, vmin=20, vmax=150)
    plt.imshow(f[2100], cmap='OrRd', alpha=0.5, vmin=20, vmax=150)
plt.xlim(300,400)
plt.ylim(400,500)
plt.show()

ModuleNotFoundError: No module named 'suite2p'

In [ ]:
#if pipeline done, look at data here

#Load 2p imaging data for each fish
"""
#Suite2p Output Files
F.npy: array of fluorescence traces (ROIs by timepoints)
Fneu.npy: array of neuropil fluorescence traces (ROIs by timepoints)
spks.npy: array of deconvolved traces (ROIs by timepoints)
stat.npy: array of statistics computed for each cell (ROIs by 1)
ops.npy: options and intermediate outputs
iscell.npy: specifies whether an ROI is a cell, first column is 0/1, and second column is probability that the ROI is a cell based on the default classifier
"""

for f in subjects:
    print(f)

    #path = '/content/drive/MyDrive/Temporary_data_storage/Matt/Dc_imaging_April2023/'+f+'/'
    path = '/content/drive/MyDrive/Personal_notebooks/David_Notebook/2p_imaging/'+f[3:9]+'/'+f+'/'

    #identify the real Z-location of each Z-plane
    zslices = [0]

    neuron_traces=[]
    neuron_coordinates=[]

    for ndx,zs in enumerate(zslices):
      if ndx>-1:
        fluor=np.load(path+"suite2p/plane"+str(ndx)+"/F.npy",allow_pickle=True)
        iscell=np.load(path+"suite2p/plane"+str(ndx)+"/iscell.npy",allow_pickle=True)
        stat=np.load(path+"suite2p/plane"+str(ndx)+"/stat.npy",allow_pickle=True)
        output_ops =np.load(path+"suite2p/plane"+str(ndx)+"/ops.npy", allow_pickle=True).item()

        for f,i,s in zip(fluor,iscell,stat):
            if i[0]==1 and np.isfinite(scipy.stats.zscore(f)).all()==True:
              neuron_traces.append(scipy.stats.zscore(f))
              neuron_coordinates.append([int(np.median(s['xpix'])),int(np.median(s['ypix'])),zs])

        """
        plt.subplot(1, 4, 1)
        plt.imshow(output_ops['refImg'], cmap='gray', )
        plt.title("Reference Image for Registration");
        plt.subplot(1, 4, 2)
        plt.imshow(output_ops['max_proj'], cmap='gray')
        plt.title("Registered Image, Max Projection");
        plt.subplot(1, 4, 3)
        plt.imshow(output_ops['meanImg'], cmap='gray')
        plt.title("Mean registered image")
        plt.subplot(1, 4, 4)
        plt.imshow(output_ops['meanImgE'], cmap='gray')
        plt.title("High-pass filtered Mean registered image");
        plt.show()

        plt.subplot(1, 2, 1)
        sns.heatmap([i for i,j in zip(neuron_traces,neuron_coordinates) if j[2]==zs], cmap="inferno", vmin=-1,vmax=4)
        plt.subplot(1, 2, 2)
        plt.imshow(output_ops['meanImgE'], cmap='gray')
        plt.scatter([i[0] for i in neuron_coordinates if i[2]==zs], [i[1] for i in neuron_coordinates if i[2]==zs], s=10, color='r', alpha=0.8)
        plt.title("with cells");
        plt.tight_layout()
        plt.show()
        """

    plt.figure(figsize=[10,3])
    plt.plot(neuron_traces[0])
    plt.show()

    plt.figure(figsize=[10,3])
    plt.plot(neuron_traces[0])
    plt.xlim(4000,7000)
    plt.show()

    #save traces and coordinates for z-brain registration (in "Registration_brains-to-atlas.ipynb")
    #np.savez(path+'neuron_details.npz', neuron_coordinates=neuron_coordinates, neuron_traces=neuron_traces)

    print("number of cells, number of timepoints:  ", np.shape(neuron_traces))

    plt.figure(figsize=[15,6])
    sns.heatmap(neuron_traces, cmap="inferno", vmin=-1,vmax=4)
    plt.title('all_cells')
    plt.show()




    #take a look at data: cluster neurons based on PCA weights

    from sklearn.decomposition import PCA, FastICA
    from sklearn.cluster import KMeans, AgglomerativeClustering

    N_components= 10
    N_clusters = 10

    PCs = {}

    pca=PCA(n_components=N_components)
    pca.fit(neuron_traces)
    weights = pca.fit_transform(neuron_traces)

    for i in range(N_components):
      PCs["pc"+str(i)] = [m[i]*n for m,n in zip(weights, neuron_traces)]

    #K-means clustering of neurons by PC weights
    kmeans_pca = KMeans(n_clusters=N_clusters, init='k-means++', random_state=5)
    kmeans_pca.fit(weights)
    labels = kmeans_pca.labels_

    #Save data and plot for each cluster
    plt.figure(figsize=[20,N_clusters*3])
    last=1
    for cl in range(N_clusters):
      clust_0 = [tr for tr,clu in zip(neuron_traces,labels) if clu==cl]

      x_0 = [x for x,clu in zip([i[0] for i in neuron_coordinates],labels) if clu==cl]
      y_0 = [y for y,clu in zip([i[1] for i in neuron_coordinates],labels) if clu==cl]

      plt.subplot(N_clusters,3,last)
      plt.title('cluster'+str(cl))
      last+=1

      plt.scatter([i[0] for i in neuron_coordinates],[i[1] for i in neuron_coordinates], cmap='Greys', s=5, alpha=0.3)
      plt.scatter(x_0,y_0, cmap='Reds', s=10, alpha=0.7)

      plt.subplot(N_clusters,3,last)
      last+=1
      sns.heatmap(clust_0,cmap="inferno", vmin=-1,vmax=3)

      plt.subplot(N_clusters,3,last)
      last+=1
      plt.plot(np.mean(clust_0, axis=0), color='k', linewidth=0.8)

    #plt.savefig(behav_path+f+"_clusters_summary.jpg", dpi=300)
    plt.show()

    c=0
    plt.figure(figsize=[60,60])
    for i in clust_0:
      if max(i)>5.0:
        c=c+2
        plt.plot(i-c, color='k', alpha=0.4, linewidth=0.2)
    plt.show()


#Now, can analyze with respect to behavioral vairables


NameError: name 'subjects' is not defined

In [ ]:
#running all in batch

from suite2p.io import BinaryFile

subjects=['DZ-231225_f2_iso6_schooling','DZ-231225_f3_iso6_schooling','DZ-231225_f4_iso6_schooling','DZ-231225_f5_iso6_schooling','DZ-231225_f7_iso6_schooling',\
          ]

for f in subjects:
    print(f)

    #path = '/content/drive/MyDrive/Temporary_data_storage/Matt/Dc_imaging_April2023/'+f+'/'
    path = '/content/drive/MyDrive/PZada_Schulze_paper/Imaging_data/Schooling/'+f[3:9]+'/'+f+'/'

    tree = ET.parse(path+"Experiment.xml")
    root = tree.getroot()

    pix_size = float(root[12].attrib['pixelSizeUM'])
    frame_rate = (root[12].attrib['frameRate'])
    frame_avg = (root[12].attrib['averageNum'])
    z_steps = 1
    z_flyback = 0
    #z_steps = (root[8].attrib['steps'])
    #z_flyback = (root[8].attrib['zStreamFrames'])
    timepoints = (root[9].attrib['timepoints'])

    print("pixels are", pix_size, "um, and volumes of ", int(z_steps)-int(z_flyback), " steps (",int(z_flyback)," flyback) aquired at ", float(frame_rate)/float(z_steps*frame_avg), " Hz, with ", timepoints, " timepoints" )

    #use xml data to set additional params
    ops['nplanes'] = int(z_steps)
    ops['ignore_flyback'] = [int(z_flyback)-1]
    ops['fs'] = float(frame_rate)/float(z_steps*frame_avg)
    ops['tau'] = 3.0 #H2B-GC6s
    ops['diameter'] = int(6.0/pix_size)

    print('cell diameter in pixels is ', ops['diameter'])


    #RUN ALL as pipeline
    #note: if registration already done, it won't re-run that

    db = {'data_path': [path],}
    print(db)

    #!mkdir path+"test/"
    output_ops = suite2p.run_s2p(ops=ops, db=db)

    print(set(output_ops.keys()).difference(ops.keys()))
    list(Path(output_ops['save_path']).iterdir())
    output_ops_file = np.load(Path(output_ops['save_path']).joinpath('ops.npy'), allow_pickle=True).item()
    output_ops_file.keys() == output_ops.keys()

    plt.figure(figsize=[10,5])
    plt.subplot(121)
    with BinaryFile(Ly=output_ops['Ly'],
                    Lx=output_ops['Lx'],
                    filename=output_ops['reg_file']) as f:
        plt.imshow(f[0], cmap='GnBu', alpha=0.5, vmin=20, vmax=150)
        plt.imshow(f[2100], cmap='OrRd', alpha=0.5, vmin=20, vmax=150)
    plt.subplot(122)
    with BinaryFile(Ly=output_ops['Ly'],
                    Lx=output_ops['Lx'],
                    filename=output_ops['reg_file']) as f:
        plt.imshow(f[0], cmap='GnBu', alpha=0.5, vmin=20, vmax=150)
        plt.imshow(f[2100], cmap='OrRd', alpha=0.5, vmin=20, vmax=150)
    plt.xlim(300,400)
    plt.ylim(400,500)
    plt.show()

    plt.figure(figsize=(4,4))
    plt.subplot(211)
    plt.plot(output_ops['yoff'][:1000])
    plt.ylabel('rigid y-offsets')
    plt.subplot(212)
    plt.plot(output_ops['xoff'][:1000])
    plt.ylabel('rigid x-offsets')
    plt.show()

    #///// checking
    zslices = [0]

    neuron_traces=[]
    neuron_coordinates=[]

    for ndx,zs in enumerate(zslices):
      if ndx>-1:
        fluor=np.load(path+"suite2p/plane"+str(ndx)+"/F.npy",allow_pickle=True)
        iscell=np.load(path+"suite2p/plane"+str(ndx)+"/iscell.npy",allow_pickle=True)
        stat=np.load(path+"suite2p/plane"+str(ndx)+"/stat.npy",allow_pickle=True)
        output_ops =np.load(path+"suite2p/plane"+str(ndx)+"/ops.npy", allow_pickle=True).item()

        for f,i,s in zip(fluor,iscell,stat):
            if i[0]==1 and np.isfinite(scipy.stats.zscore(f)).all()==True:
              neuron_traces.append(scipy.stats.zscore(f))
              neuron_coordinates.append([int(np.median(s['xpix'])),int(np.median(s['ypix'])),zs])

        #"""
        plt.subplot(1, 4, 1)
        plt.imshow(output_ops['refImg'], cmap='gray', )
        plt.title("Reference Image for Registration");
        plt.subplot(1, 4, 2)
        plt.imshow(output_ops['max_proj'], cmap='gray')
        plt.title("Registered Image, Max Projection");
        plt.subplot(1, 4, 3)
        plt.imshow(output_ops['meanImg'], cmap='gray')
        plt.title("Mean registered image")
        plt.subplot(1, 4, 4)
        plt.imshow(output_ops['meanImgE'], cmap='gray')
        plt.title("High-pass filtered Mean registered image");
        plt.show()

        plt.subplot(1, 2, 1)
        sns.heatmap([i for i,j in zip(neuron_traces,neuron_coordinates) if j[2]==zs], cmap="inferno", vmin=-1,vmax=4)
        plt.subplot(1, 2, 2)
        plt.imshow(output_ops['meanImgE'], cmap='gray')
        plt.scatter([i[0] for i in neuron_coordinates if i[2]==zs], [i[1] for i in neuron_coordinates if i[2]==zs], s=10, color='r', alpha=0.8)
        plt.title("with cells");
        plt.tight_layout()
        plt.show()
        #"""

    print("number of cells, number of timepoints:  ", np.shape(neuron_traces))

    plt.figure(figsize=[15,6])
    sns.heatmap(neuron_traces, cmap="inferno", vmin=-1,vmax=4)
    plt.title('all_cells')
    plt.show()

ModuleNotFoundError: No module named 'suite2p'